In [1]:
from dotenv import load_dotenv 
from openai import OpenAI
from testUtils import save_pickle,load_pickle
load_dotenv()
client = OpenAI()

## text generation/chat

```python
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "which city is China's Capital?"
        }
    ]
)

save_pickle(completion,'completion-test1.pkl')
```

In [2]:
completion = load_pickle('completion-test1.pkl')

In [3]:
print(completion.choices[0].message)

ChatCompletionMessage(content='The capital of China is Beijing.', refusal=None, role='assistant', function_call=None, tool_calls=None)


### Returned object

- Type of `ChatCompletion`
- https://platform.openai.com/docs/api-reference/chat/object

In [4]:
type(completion)

openai.types.chat.chat_completion.ChatCompletion

In [5]:
from testUtils import print_properties
print_properties(completion)

id:
  chatcmpl-AJDBs7ryJnfpfJDLg6xg2F9FFn0RD
choices:
  [0]:
    finish_reason:
      stop
    index:
      0
    logprobs:
      None
    message:
      content:
        The capital of China is Beijing.
      refusal:
        None
      role:
        assistant
      function_call:
        None
      tool_calls:
        None
created:
  1729142300
model:
  gpt-4o-mini-2024-07-18
object:
  chat.completion
service_tier:
  None
system_fingerprint:
  fp_e2bde53e6e
usage:
  completion_tokens:
    7
  prompt_tokens:
    23
  total_tokens:
    30
  completion_tokens_details:
    audio_tokens:
      None
    reasoning_tokens:
      0
  prompt_tokens_details:
    audio_tokens:
      None
    cached_tokens:
      0
_request_id:
  req_ed3d3ebdbc6c119a6eb99185f09b34fd


In [6]:
import json
completion_dict = completion.to_dict()
print(json.dumps(completion_dict, indent=2))

{
  "id": "chatcmpl-AJDBs7ryJnfpfJDLg6xg2F9FFn0RD",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The capital of China is Beijing.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1729142300,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_e2bde53e6e",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 23,
    "total_tokens": 30,
    "completion_tokens_details": {
      "reasoning_tokens": 0
    },
    "prompt_tokens_details": {
      "cached_tokens": 0
    }
  }
}


### manage output tokens and Context

In [7]:
import tiktoken
tokenizer = tiktoken.encoding_for_model("gpt-4o-mini")

# Function to calculate tokens in messages
def count_tokens(messages):
    return sum(len(tokenizer.encode(message["content"])) for message in messages)

# Function to manage context and limit tokens
def generate_response(messages, max_output_tokens=100, max_context_tokens=3000):
    # Calculate total tokens used in current context
    total_tokens = count_tokens(messages)

    # Truncate context if it exceeds max_context_tokens
    while total_tokens > max_context_tokens:
        # Remove the oldest user/assistant messages until within the limit
        messages.pop(1)  # Popping after system message
        total_tokens = count_tokens(messages)

    # Call the OpenAI API with controlled max_tokens
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=max_output_tokens
    )

    return completion

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Which city is China's capital?"}
]

# completion = generate_response(messages, max_output_tokens=100, max_context_tokens=3000)
# save_pickle(completion,'completion-test2.pkl')

In [8]:
load_pickle('completion-test2.pkl')
print(completion.choices[0].message)

ChatCompletionMessage(content='The capital of China is Beijing.', refusal=None, role='assistant', function_call=None, tool_calls=None)
